##### 导入库

In [29]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import tree
from collections import Counter
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE,SMOTE
from sklearn.model_selection import GridSearchCV
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn

##### 选择数据

In [30]:
n='n.csv'

p={
    "all":'all.csv',
    "cns":'cns.csv',
    "opi":'opi.csv',
    "other":'other.csv'
}

way="other"


##### 读取数据

In [31]:
fea_dict={
    'all':[
"fft_imag_3",
"fft_angle_3",
"alt_stderr_min_50",
"alt_rvalue_min_50",
"fft_real_2",
"fft_real_5",
"erbc_num10_focus2",
"fft_imag_6",
"lumpiness"
],


 'opi':[
"fft_imag_3",
"erbc_num10_focus4",
"alt_rvalue_min_50",
"fft_angle_3",
"erbc_num10_focus2",
"fft_real_5"
],

 'cns':[
"fft_imag_3",
"fft_angle_3",
"fft_real_5",
"minindex",
"lumpiness",
"fft_imag_6",
"alt_rvalue_min_50",
"fft_angle_2",
"erbc_num10_focus2",
"S_down",
"retindex",
"erbc_num10_focus4",
"mac",
"alt_stderr_min_50",
],

'other':[
  "fft_imag_3",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_imag_6",
  "fft_real_2",
  "alt_rvalue_min_50",
  "fft_real_5",
  "minvalue",
  "V2",
  "F2",
  "retindex"
]
}

fea_list=fea_dict[way]

def read_data(tit):
    path = 'E:\\desktop\\data\\app\\223\\fea\\ok\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data

n_fea=read_data(n)
p_fea=read_data(p[way])

n_fea["tag"]=[0 for i in range(len(n_fea))]
p_fea["tag"]=[1 for i in range(len(p_fea))]

print(len(n_fea),len(p_fea))

count_fea=[]
for key in fea_dict.keys():
    count_fea=count_fea+fea_dict[key]

count_fea=set(count_fea)

521 90


##### 过采样

In [32]:
#过采样
over=SMOTE(random_state=0)
# over=SVMSMOTE(random_state=0)
np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))
x_data_=np_fea[fea_list]
y_data_=np_fea["tag"]

x_data,y_data=over.fit_resample(x_data_,y_data_)

#切分
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3)
print(x_data.shape)

(1042, 11)


In [33]:
def jugde(y_pred,y_true):
    tn,fn,tp,fp=0,0,0,0
    for i in range(len(y_pred)):
        
        if y_pred[i]==y_true[i]:
            if y_true[i]:
                tp+=1
            else:
                tn+=1
        else:
            if y_true[i]:
                fp+=1
            else:
                fn+=1

    print("检出率： ",tp,"/",tp+fp,' ',round(tp*100/(tp+fp),2),"%")
    print("误判率： ",fn,"/",tn+fn,' ',round(fn*100/(tn+fn),2),"%")
    print("精确率： ",(tn+tp),"/",(tn+fp+tp+fp),' ',round((tn+tp)*100/(tn+fp+tp+fp),2),"%")


##### 搭建模型

In [34]:
etc=ExtraTreesClassifier(random_state=0)
etc.fit(x_train,y_train)
y_pred=etc.predict(x_test)

print("测试集中：")
jugde(y_pred,list(y_test))

print("所有数据中：")
y_pred=etc.predict(x_data_)
jugde(y_pred,list(y_data_))

etc.score(x_test,y_test)

测试集中：
检出率：  142 / 148   95.95 %
误判率：  14 / 165   8.48 %
精确率：  293 / 305   96.07 %
所有数据中：
检出率：  88 / 90   97.78 %
误判率：  14 / 521   2.69 %
精确率：  595 / 599   99.33 %


0.9361022364217252

In [35]:
#保存模型
print(len(fea_list))
initial_type = [('float_input', FloatTensorType([None,  x_train.shape[1]]))]
onx = convert_sklearn(etc, initial_types=initial_type,
                      target_opset=12)

with open("./mod/"+way+".onnx", "wb") as f:
    f.write(onx.SerializeToString())

print(way+"保存成功")

11
